<a href="https://colab.research.google.com/github/RE-NY/Defect-detection-from-drone-images-of-Wind-turbine/blob/main/Drone_inspection_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
from time import time
from time import sleep

In [ ]:
def give_dataset(folder):
  onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

  print("Working with {0} images".format(len(onlyfiles)))
  #print("Image examples: ")

  #for i in range(2):
  #   print(onlyfiles[i])
    #  display(_Imgdis(filename=folder + "/" + onlyfiles[i], width=240, height=320))
  onlyfiles[1]
  type(onlyfiles[1])

  from scipy import ndimage
  from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

  train_files = []
  i=0
  for _file in onlyfiles:
      train_files.append(_file)
      label_in_file = _file.find("_")
  print("Files in train_files: %d" % len(train_files))

  # Original Dimensions      #2970 #5280 #3
  image_width = 5280
  image_height = 2970
  ratio = 4

  image_width = int(image_width / ratio) - 1
  image_height = int(image_height / ratio)

  channels = 3
  nb_classes = 1

  dataset = np.ndarray(shape=(len(train_files), image_width , image_height , channels),
                      dtype=np.float32)

  i = 0
  for _file in train_files:
      img = load_img(folder + "/" + _file)  # this is a PIL image
      img.thumbnail((image_width, image_height))
      # Convert to Numpy Array
      x = img_to_array(img)
      x = x.reshape((1319 , 742 , 3))
      # Normalize
      x = (x - 128.0) / 128.0
      dataset[i] = x
      i += 1
      if i % 250 == 0:
          print("%d images to array" % i)
  #print("All images to array!")
  return dataset

In [ ]:
#dataset = give_dataset("/content/drive/MyDrive/Image folder")
#print((dataset))

In [ ]:
#making X_train.......
'''
X_train_correct = dataset_correct
X_train_defected = dataset_defected
X_train = sum of those...

y_train_correct = np.full((1,#total examples) , 1 , dtype = int)

'''



'\nX_train_correct = dataset_correct\nX_train_defected = dataset_defected\nX_train = sum of those...\n\ny_train_correct = np.full((1,#total examples) , 1 , dtype = int)\n\n'

In [ ]:
X_train_correct = give_dataset("/content/drive/MyDrive/Correct")

Working with 40 images
Files in train_files: 40


In [ ]:
X_train_defected = give_dataset("/content/drive/MyDrive/Defected")

Working with 40 images
Files in train_files: 40


In [ ]:
y_train_correct = np.full((1 , len(X_train_correct)) , 1)
y_train_defected = np.full((1 , len(X_train_correct)) , 0)


In [ ]:
#concatenation....
X_train = np.concatenate((X_train_correct , X_train_defected) , axis = 0)



In [ ]:
y_train = np.concatenate((y_train_correct , y_train_defected) , axis = 1)

In [ ]:
y_train = y_train.reshape((80,1))
y_train

ValueError: ignored

In [ ]:
#Using the split to make final data.....
from sklearn.model_selection import train_test_split

#Splitting
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=33)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=33)
print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))

Train set size: 64, Val set size: 8, Test set size: 8


In [ ]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten
from keras import backend as k

inpx = Input(shape=(1319 , 742 , 3))
layer1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inpx)
layer2 = Conv2D(32, (3, 3), activation='relu')(layer1)
layer3 = MaxPooling2D(pool_size=(3, 3))(layer2)
layer4 = Dropout(0.5)(layer3)
layer5 = Flatten()(layer4)
#layer6 = Dense(512, activation='relu')(layer5)
layer6 = Dense(1 , activation='sigmoid')(layer5)

model = Model([inpx], layer6)
model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1319, 742, 3)]    0         
                                                                 
 conv2d_2 (Conv2D)           (None, 1317, 740, 32)     896       
                                                                 
 conv2d_3 (Conv2D)           (None, 1315, 738, 32)     9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 438, 246, 32)      0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 438, 246, 32)      0         
                                                                 
 flatten_1 (Flatten)         (None, 3447936)           0         
                                                           

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(64, 1319, 742, 3)
(64, 1)
(8, 1319, 742, 3)
(8, 1)


In [ ]:
model.fit(X_train, y_train, epochs=50 ,batch_size = 16, validation_data = (X_val , y_val))

Epoch 1/50
4/4 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.5469

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


4/4 [==============================] - 20s 2s/step - loss: 0.0000e+00 - accuracy: 0.5469 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 2/50
4/4 [==============================] - 4s 926ms/step - loss: 0.0000e+00 - accuracy: 0.5156 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/50
4/4 [==============================] - 4s 919ms/step - loss: 0.0000e+00 - accuracy: 0.5156 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/50
4/4 [==============================] - 4s 964ms/step - loss: 0.0000e+00 - accuracy: 0.5156 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 5/50
4/4 [==============================] - 4s 927ms/step - loss: 0.0000e+00 - accuracy: 0.5156 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 6/50
4/4 [==============================] - 4s 936ms/step - loss: 0.0000e+00 - accuracy: 0.5156 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 7/50
4/4 [==============================] - 4s 931ms/step - loss: 0.0000e+00 - accuracy: 0.5156 - val_loss: 0.0000e+00 

In [ ]:
results = model.evaluate(X_test , y_test , batch_size = 4)
results

2/2 [==============================] - 2s 46ms/step - loss: 0.0000e+00 - accuracy: 0.3750


[0.0, 0.375]

In [ ]:
#For Data augmentation..............
# Importing necessary functions
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img

# Initialising the ImageDataGenerator class.
# We will pass in the augmentation parameters in the constructor.
datagen = ImageDataGenerator(
    horizontal_flip = True,
		rotation_range = 40,
		shear_range = 0.2,
		zoom_range = 0.2,
		brightness_range = (0.5, 1.5))

# Loading a sample image
img = load_img('/content/drive/MyDrive/Correct/DJI_0590.JPG')
# Converting the input sample image to an array
x = img_to_array(img)
# Reshaping the input image
x = x.reshape((1, ) + x.shape)

# Generating and saving 5 augmented samples
# using the above defined parameters.
i = 0
for batch in datagen.flow(x, batch_size = 1,
						save_to_dir ='/content/drive/MyDrive/Colab Notebooks',
						save_prefix ='image', save_format ='JPG'):
	i += 1
	if i > 2: # gives three including itself.....
		break
